In [1]:
from MetaCriticScraper import MetaCriticScraper

In [3]:
import urllib.request
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd

years = [i for i in range(2010, 2024)]

for y in years:
    year = str(y)
    hrd = {'User-Agent' : 'Mozilla/5.0', 'referer' : 'https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=' + year + '&distribution=&sort=desc&view=detailed'}
    url = "https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=" + year + "&distribution=&sort=desc&view=detailed"

    req = requests.get(url, headers=hrd)
    # response = urllib.request.urlopen(url)

    soup = BeautifulSoup(req.content, "html.parser", from_encoding='utf-8')
    # results = soup.select("a")  #id 아래에 strong을 가져오겠다
    questions = soup.find("div", {"class":"title_bump"})

    d = list(questions.find_all("a", class_="page_num"))
    # num_list = [str(int(item.string.strip())-1) for item in d]
    url_num_list = [int(item.string.strip()) for item in d]

    num_list = []
    for i in range(url_num_list[len(url_num_list)-1]):
        num_list.append(str(i))

    game_urls = []
    for j in range(len(num_list)):
        hrd = {'User-Agent' : 'Mozilla/5.0', 'referer' : "https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=" + year + "&distribution=&sort=desc&view=detailed" + "&page=" + num_list[j] }
        url = "https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=" + year + "&distribution=&sort=desc&view=detailed" + "&page=" + num_list[j]

        req = requests.get(url, headers=hrd)
        # response = urllib.request.urlopen(url)

        soup = BeautifulSoup(req.content, "html.parser", from_encoding='utf-8')
        # results = soup.select("a")  #id 아래에 strong을 가져오겠다
        # 게임 목록을 가져오기 위해 필요한 태그와 클래스를 확인합니다.
        game_items = soup.find_all("td", class_="clamp-summary-wrap")

        # 각 게임별 URL을 추출합니다.
        
        for item in game_items:
            link = item.find("a", class_="title")
            game_urls.append(link['href'])

        df = pd.DataFrame(columns=['name', 'platform','publisher', 'rating', 'genre', '#_of_player', 'release_date', 'critic_score', 'critic_count', 'user_score', 'user_count','img_url', 'description'])

    for i in game_urls:
        url = 'https://www.metacritic.com' + i
        scraper = MetaCriticScraper(url)
        df = pd.concat([df, pd.DataFrame([{'name' : scraper.game['title'], 'platform': scraper.game['platform'], 'publisher':scraper.game['publisher'],
                            'rating': scraper.game['rating'] , 'genre':scraper.game['genre'] , 
                            '#_of_player':scraper.game['nplayer'] , 'release_date':scraper.game['release_date'] , 'critic_score':scraper.game['critic_score'] , 
                            'critic_count':scraper.game['critic_count'], 'user_score':scraper.game['user_score'] , 'user_count':scraper.game['user_count'], 
                            'img_url':scraper.game['image'] , 'description':scraper.game['description']}])], ignore_index=True)
    
    filename = year + '_raw.csv'  # 파일 이름 생성
    df.to_csv('./data/' + filename, index=False)  
    